# Building Predictive Models

In [9]:
import pandas as pd
import os
import numpy as np

## Import Data

In [10]:
#set the path of the processed data
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path,'train.csv')
test_file_path = os.path.join(processed_data_path,'test.csv')

In [11]:
#create train and test dataframes using pandas
train_df = pd.read_csv(train_file_path,index_col='PassengerId')
test_df = pd.read_csv(test_file_path,index_col='PassengerId')

In [1]:
train_df.columns()

NameError: name 'train_df' is not defined

In [12]:
#See info on dataframes to check if everything is ok
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_Z              891 non-null    int64  
 14  Pclass_1            891 non-null    int64  
 15  Pclass_2            891 non-null    int64  
 16  Pclass_3

In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    int64  
 6   Deck_B              418 non-null    int64  
 7   Deck_C              418 non-null    int64  
 8   Deck_D              418 non-null    int64  
 9   Deck_E              418 non-null    int64  
 10  Deck_F              418 non-null    int64  
 11  Deck_G              418 non-null    int64  
 12  Deck_Z              418 non-null    int64  
 13  Pclass_1            418 non-null    int64  
 14  Pclass_2            418 non-null    int64  
 15  Pclass_3            418 non-null    int64  
 16  Title

 ## Data Preparation

In [16]:
X = train_df.loc[:,'Age':].to_numpy().astype('float') #input matrix
y = train_df['Survived'].ravel() #output array

In [18]:
print (X.shape, y.shape)

(891, 32) (891,)


In [19]:
#ML train test split 80/20 (80% will be used to train, 20% to test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [21]:
#average survival in train and test, without ML 
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


## Baseline Model

In [22]:
import sklearn

In [24]:
#import function
from sklearn.dummy import DummyClassifier

In [25]:
#create a model
model_dummy = DummyClassifier(strategy='most_frequent',random_state=0)

In [26]:
#train a model
model_dummy.fit(X_train,y_train) #input,output params are necessary for fit training function

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [27]:
#calculate model score (baseline considers most frequent data, a.k.a 0 == non survived titanic)
print('score for baseline model : {0:.2f}'.format(model_dummy.score(X_test,y_test)))

score for baseline model : 0.61


In [28]:
#So, without any ML algorithm, if we simply predict that a passenger in the titanic disaster dies, we will be right in 61% of the cases
#let's try to improve that with ML! 
#First, lets import some performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [29]:
#accuracy score
print('accuracy for baseline model : {0:.2f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

accuracy for baseline model : 0.61


In [31]:
#confusion matrix
print('confusion matrix for baseline model : \n {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

confusion matrix for baseline model : 
 [[110   0]
 [ 69   0]]


In [32]:
#precision and recall scores
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test,model_dummy.predict(X_test))))
print('recall for baseline model : {0:.2f}'.format(recall_score(y_test,model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model : 0.00


/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## First Kaggle Submission - Baseline Data

In [33]:
#converting to the matrix
test_X = test_df.to_numpy().astype('float')

In [34]:
#get predictions
predictions = model_dummy.predict(test_X)

In [37]:
#submission dataframe
df_submission = pd.DataFrame({'PassengerId' : test_df.index, 'Survived' : predictions})

In [38]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [39]:
#submission file
submission_data_path = os.path.join(os.path.pardir,'data','external')
submission_file_path = os.path.join(submission_data_path,'01_dummy.csv')

In [40]:
#write to the file
df_submission.to_csv(submission_file_path,index=False)

In [43]:
def get_submission_file(model,filename):
    #converting to the matrix
    test_X = test_df.to_numpy().astype('float')
    #get predictions
    predictions = model.predict(test_X)
    #submission dataframe
    df_submission = pd.DataFrame({'PassengerId' : test_df.index, 'Survived' : predictions})
    #submission file
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path,filename)
    #write to the file
    df_submission.to_csv(submission_file_path,index=False)
    

In [44]:
#use the method we created above with all steps to get the submission file
get_submission_file(model_dummy,'01_dummy.csv')

## Logistic Regression Model

In [45]:
#import logistic regression function from sklearn
from sklearn.linear_model import LogisticRegression

In [46]:
#create our model 
model_lr_1 = LogisticRegression(random_state=0)

In [47]:
#train our model
model_lr_1.fit(X_train,y_train)

/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
#evaluate model
print('score for logistic regression - version 1 : {0:.2f}'.format(model_lr_1.score(X_test,y_test)))

score for logistic regression - version 1 : 0.83


In [49]:
#performance metrics

#accuracy score
print('accuracy for logistic regression v1 model : {0:.2f}'.format(accuracy_score(y_test,model_lr_1.predict(X_test))))
#confusion matrix
print('confusion matrix for logistic regression v1 model : \n {0}'.format(confusion_matrix(y_test,model_lr_1.predict(X_test))))
#precision 
print('precision for logistic regression v1 model : {0:.2f}'.format(precision_score(y_test,model_lr_1.predict(X_test))))
#recall
print('recall for logistic regression v1 model : {0:.2f}'.format(recall_score(y_test,model_lr_1.predict(X_test))))

accuracy for logistic regression v1 model : 0.83
confusion matrix for logistic regression v1 model : 
 [[95 15]
 [15 54]]
precision for logistic regression v1 model : 0.78
recall for logistic regression v1 model : 0.78


In [50]:
#model coefficients
model_lr_1.coef_

array([[-0.02839123,  0.00360907, -0.51078071,  0.38069288, -0.75868441,
         0.13981015,  0.02514833, -0.2482328 ,  0.40853622,  0.85403118,
         0.20433994, -0.01351029, -0.28121413,  0.9417177 ,  0.48676767,
        -0.33957676,  0.07786513,  0.95158021,  0.54316116, -1.50753011,
         1.22656673, -0.07084393, -0.13189058,  0.14793129,  0.15729772,
         0.28313474,  0.50054487,  0.48355505,  0.45137928,  0.15397428,
         0.30511452,  0.78379409]])

## Second Kaggle Submission

In [51]:
#get submission file
get_submission_file(model_lr_1,'02_lr.csv')

# Part 2

## Hyperparameter Optimization

In [52]:
# base model
model_lr = LogisticRegression(random_state=0)

In [54]:
from sklearn.model_selection import GridSearchCV

In [56]:
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)

In [57]:
clf.fit(X_train,y_train)

/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: Convergence

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [58]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [59]:
print('best score: {0:.2f}'.format(clf.best_score_))

best score: 0.83


In [60]:
#evaluate model
print('score for logistic regression v2 : {0:.2f}'.format(clf.score(X_test,y_test)))

score for logistic regression v2 : 0.83


## Making thrird Kaggle Submission

In [62]:
#get submission file
get_submission_file(clf,'04_lr.csv')

## Feature Normalization and Standardization

In [63]:
#import functions
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### Feature Normalization - put features in similar scale!

In [64]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [65]:
#check in one column the min and max of our scale
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [66]:
#normalize test data
X_test_scaled = scaler.transform(X_test)

### Feature Standardization

In [67]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create Model after standardization

In [68]:
#base model
model_lr = LogisticRegression(random_state=0)
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)
clf.fit(X_train_scaled,y_train)

/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/gutospinelli/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitF

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [69]:
print('best score: {0:.2f}'.format(clf.best_score_))

best score: 0.81


In [71]:
#evaluate model
print('score for logistic regression v2 - standardized : {0:.2f}'.format(clf.score(X_test_scaled,y_test)))

score for logistic regression v2 - standardized : 0.84


## Model Persistence

In [72]:
#import pickle lib to persist our model to the disk
import pickle

In [73]:
#create the file path
model_file_path = os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir,'models','lr_scaler.pkl')

In [74]:
#open the files to write
model_file_pickle = open(model_file_path,'wb') #wb = write bytes
scaler_file_pickle = open(scaler_file_path,'wb')

In [75]:
#persist model and scaler to the disk using pickle
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [76]:
#close the files
model_file_pickle.close()
scaler_file_pickle.close()

### load the persisted file

In [78]:
#open the files in read mode
model_file_pickle = open(model_file_path,'rb') #rb = read bytes
scaler_file_pickle = open(scaler_file_path,'rb')
#load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
#close files
model_file_pickle.close()
scaler_file_pickle.close()

In [79]:
clf_loaded

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [80]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [82]:
#transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
#calculate the score using loaded model object
print('score for persisted logistic regression : {0:.2f}'.format(clf_loaded.score(X_test_scaled,y_test)))

score for persisted logistic regression : 0.84
